In [1]:
import pandas as pd
import numpy as np

In [2]:
def read_transform(file, cols, zone_df):
    """ Función que nos lee el file indicado usando las columnas deseadas y utiliza el dataframe zone_df
    para hacer ciertas transformaciones en el dataframe final deseado"""

    
    #     Leemos el archivo con las columnas deseadas
    df = pd.read_csv(file,usecols=cols, parse_dates=[0,1])
   #     Nos quedamos con 120k líneas al azar y yambiamos el nombre a las columnas por unos más 
   #     intuitivos y más cortos
    df = df.sample(120000)
    
    df = df.rename(
    columns={"tpep_pickup_datetime" : "PU_datetime" 
            , "tpep_dropoff_datetime" : "DO_datetime"
            , "trip_distance" : "distance_miles"
            , "PULocationID" : "PULocID"
            , "DOLocationID" : "DOLocID"
            , "tip_amount" : "tip"
            , "total_amount" : "total"})
    
#     Queremos hacer clasificaciones en función del barrio en el que surjan y acaben los viajes, por lo que
#     añadimos estas dos columnas, de momento vacías
    df['PUBorough'] = np.nan
    df['DOBorough'] = np.nan
    
#    En el dataframe con las zonas hemos eliminado un par de IDs, por lo que debemos asegurarnos que en el 
#    dataframe con la info de los viajes no los contenga. Para ello creamos unos set de comparación con las 
#    columnas que contienen los ID
    zones_set = set(zone_df.LocationID.values)
    PU_set = set(df.PULocID.values)
    DO_set = set(df.DOLocID.values)
    
#     Mediante los dos siguientes bucles quitamos los IDs que están en el dataframe de los viajes que no 
#     estén en el DataFrame con las zonas. Uno es para el origen de los viajes y otro para el fin
    for val in PU_set.difference(zones_set):
        df = df.loc[df['PULocID'] != val]
        
    for val in DO_set.difference(zones_set):
        df = df.loc[df['DOLocID'] != val]
        
#     Ya podemos rellenar las columnas de los barrios en el dataframe de los viajes, para ellos buscamos 
#     coicidencias en el ID entre los dos dataframes que tenemos y vamos metiendo valores
    for val in set(df['PULocID'].values):
        df.loc[df['PULocID'] == val, 'PUBorough'] = zone_df.loc[zone_df['LocationID'] == val, 'Borough'].values[0]
    
    for val in set(df['DOLocID'].values):
        df.loc[df['DOLocID'] == val, 'DOBorough'] = zone_df.loc[zone_df['LocationID'] == val, 'Borough'].values[0]
        
#     Nos quedaremos con el momemnto en el que se inicia el viaje y la duración del viaje. Esta última 
#     columna la creamos restando final - inicio y haciendo las transformaciones necesarias
    df['trip_duration_min'] = (df.DO_datetime - df.PU_datetime).astype('timedelta64[s]')
    df['trip_duration_min'] = round(df['trip_duration_min']/60,2)
    
#    Mi intención es quedarme con aquellos viajes cuyo origen y destinto están en el mismo barrio para 
#    comprobar las hipótesis deseadas

#     df = df.loc[df['PUBorough'] == df['DOBorough']] 
#   Parece que tarda algo menos la negación de la desigualdad así que nos quedamos con ella
    df = df.loc[~(df['PUBorough'] != df['DOBorough'])]
    
#   Como nos hemos quedado con los viajes cuyo inicio y fin son en el mismo barrio, podemos borrar una 
#   columna y así ahorramos algo de espacio. En la misma línea, eliminamos la columna con el tiempo en el
#   que se termina el viaje
    df = df.rename(columns={"PUBorough" : "Borough" })
    df = df[['PU_datetime','Borough','trip_duration_min','distance_miles','tip','total']]
    
    return df

In [3]:
# Primero leemos el archivo que contiene las correspondencias entre los ID y los barrios de Nueva York
zones = pd.read_csv("zones.csv")

# Eliminamos aquellas filas que tienen un barrio desconocido o que no está entre los 5 conocidos
zones = zones[(zones.Borough != "EWR") & (zones.Borough != "Unknown")]

# Esta es la lista de columnas que queremos conservar del archivo que leemos
columns_desired = ["tpep_pickup_datetime", "tpep_dropoff_datetime", "trip_distance",
                   "PULocationID",	"DOLocationID", "tip_amount", "total_amount"]	
    
files = ['jan_2019.csv', 'apr_2019.csv', 'jul_2019.csv', 'oct_2019.csv']

In [4]:
Jan = read_transform('jan_2019.csv',columns_desired, zones)
Feb = read_transform('feb_2019.csv',columns_desired, zones)
Mar = read_transform('mar_2019.csv',columns_desired, zones)
Apr = read_transform('apr_2019.csv',columns_desired, zones)
May = read_transform('may_2019.csv',columns_desired, zones)
Jun = read_transform('jun_2019.csv',columns_desired, zones)
Jul = read_transform('jul_2019.csv',columns_desired, zones)
Aug = read_transform('aug_2019.csv',columns_desired, zones)
Sep = read_transform('sep_2019.csv',columns_desired, zones)
Oct = read_transform('oct_2019.csv',columns_desired, zones)
Nov = read_transform('nov_2019.csv',columns_desired, zones)
Dec = read_transform('dec_2019.csv',columns_desired, zones)
taxi_2019 = pd.concat([Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec], ignore_index=True)

In [5]:
q1 = len(Jan)+len(Feb)+len(Mar)
q2 = len(Apr)+len(May)+len(Jun)
q3 = len(Jul)+len(Aug)+len(Sep)
q4 = len(Oct)+len(Nov)+len(Dec)

In [6]:
Q1 = taxi_2019.iloc[range(0,q1),:]
Q2 = taxi_2019.iloc[range(q1,q1+q2),:]
Q3 = taxi_2019.iloc[range(q1+q2,q1+q2+q3),:]
Q4 = taxi_2019.iloc[range(q1+q2+q3,q1+q2+q3+q4),:]

In [7]:
Q1.to_csv('Q1.csv',header=True,index=True)
Q2.to_csv('Q2.csv',header=True,index=True)
Q3.to_csv('Q3.csv',header=True,index=True)
Q4.to_csv('Q4.csv',header=True,index=True)
taxi_2019.to_csv('taxi_2019.csv',header=False)